In [47]:
from dataclasses import dataclass, field
from functools import cache
from typing import TypeVar, Sequence, FrozenSet, Optional, Collection, Set, Union, Tuple, Iterator

import more_itertools


In [48]:
_U = TypeVar('_U')
_ForwardFrozenRelation = TypeVar('_ForwardFrozenRelation', bound='FrozenRelation')


@dataclass(frozen=True)
class FrozenRelation:
    relation_set: FrozenSet[tuple] = field(default_factory=frozenset)

    def __getitem__(self, item):
        return self.get(item)

    def __str__(self):
        return "{}{}{}".format('{', ",".join(map(lambda e: "({})".format(','.join(map(str, e))), self.relation_set)), '}')

    def related(self, item1, item2) -> bool:
        return (item1, item2) in self.relation_set

    def get(self, item) -> Set[_U]:
        return {elem[1] for elem in self.relation_set if elem[0] == item}

    def get_inverse(self, item) -> Set[_U]:
        return {elem[0] for elem in self.relation_set if elem[1] == item}

    @staticmethod
    def from_ground_coll(ground_coll: Collection[tuple],
                         transitive: bool = False,
                         reflexive: bool = False,
                         symmetric: bool = False) -> _ForwardFrozenRelation:
        relation_set = set(ground_coll)
        add = {None}
        while add:
            add = set()
            if symmetric:
                for rel in relation_set:
                    item1, item2 = rel
                    if (item2, item1) not in relation_set:
                        add.add((item2, item1))
            if reflexive:
                for rel in relation_set:
                    item1, item2 = rel
                    if (item1, item1) not in relation_set:
                        add.add((item1, item1))
                    if (item2, item2) not in relation_set:
                        add.add((item2, item2))
            if transitive:
                for rel in relation_set:
                    item1, item2 = rel
                    for rel_ in relation_set:
                        item3, item4 = rel_
                        if item2 != item3:
                            continue
                        if (item1, item4) not in relation_set:
                            add.add((item1, item4))
            relation_set.update(add)
        return FrozenRelation(frozenset(relation_set))


In [49]:
_ForwardAtom = TypeVar('_ForwardAtom', bound='Atom')
_ForwardFormula = TypeVar('_ForwardFormula', bound='Formula')


@dataclass(order=True, frozen=True)
class Atom:
    symbol: str
    arguments: Sequence[Union[_ForwardAtom, _ForwardFormula]] = field(default_factory=tuple)

    @property
    def is_complement(self) -> bool:
        return self.symbol.startswith('-')

    @property
    def is_know(self) -> bool:
        return self.symbol == 'Know'

    def __invert__(self):
        if self.is_complement:
            return Atom(self.symbol[1:], self.arguments)
        else:
            return Atom('-{}'.format(self.symbol), self.arguments)

    def __str__(self):
        if self.arguments:
            return "{}({})".format(self.symbol, ','.join(map(str, self.arguments)))
        else:
            return "{}".format(self.symbol)


In [50]:
@dataclass(order=True, frozen=True)
class Literal:
    atom: Atom
    sign: bool = field(default=True)

    def __neg__(self):
        return Literal(self.atom, not self.sign)

    def __abs__(self):
        return Literal(self.atom)

    def __invert__(self):
        return Literal(~self.atom, self.sign)

    def __str__(self):
        if self.sign:
            return str(self.atom)
        else:
            return "¬{}".format(self.atom)

In [51]:
Action = Atom
Fluent = Atom
FluentLiteral = Literal

In [52]:
_ForwardCausalSetting = TypeVar('_ForwardCausalSetting', bound='CausalSetting')

In [53]:
@dataclass(order=True, frozen=True)
class State:
    world: FrozenSet[FluentLiteral] = field(default_factory=frozenset)
    k_relation: FrozenRelation = field(default_factory=FrozenRelation)
    prioritized_goals: Sequence[_ForwardFormula] = field(default_factory=tuple)

    def __contains__(self, item):
        if not isinstance(item, FluentLiteral):
            raise TypeError("Unexpected type {} of item {}".format(type(item).__name__, item))
        if item in self.world:
            return True
        if -item in self.world:
            return False
        return not item.sign


In [54]:
_ForwardSituation = TypeVar('_ForwardSituation', bound='Situation')


@dataclass(order=True, frozen=True)
class Situation:
    state: State = field(default_factory=State)
    action: Optional[Action] = field(default=None)
    parent: Optional[_ForwardSituation] = field(default=None, repr=False)

    @property
    def is_root(self) -> bool:
        return self.parent is not None

    @property
    def time(self) -> int:
        if self.is_root:
            return 0
        else:
            return self.parent.time + 1

    @property
    def root(self) -> _ForwardSituation:
        if self.is_root:
            return self
        else:
            return self.parent.root

In [55]:
_ForwardPath = TypeVar('_ForwardPath', bound='Path')


@dataclass(order=True, frozen=True)
class Path:
    states: Sequence[State] = field(default_factory=tuple)
    actions: Sequence[Action] = field(default_factory=tuple)

    @property
    def root(self) -> State:
        return self.states[0]

    def time(self, state: State) -> Optional[int]:
        for time, state_ in enumerate(self.states):
            if state_ == state:
                return time
        return None

    def starts(self, state: State) -> bool:
        return self.root == state

    def get_suffix(self, state: Optional[State] = None) -> _ForwardPath:
        if state is None:
            state = self.root
        if state not in self.states:
            return Path()
        offset = self.time(state)
        if offset + 1 < len(self.actions):
            return Path(self.states[offset:], self.actions[offset + 1:])
        else:
            return Path(self.states[offset:])


In [56]:
class Formula:
    def __neg__(self):
        return Negation(self)

    def __and__(self, other):
        return Conjunction(self, other)

    def __or__(self, other):
        return Disjunction(self, other)

    def evaluate(self, causal_setting: _ForwardCausalSetting, elem: Union[Situation, Path]) -> bool:
        if isinstance(elem, Situation):
            return self.evaluate_situation(causal_setting, elem)
        else:
            if not isinstance(elem, Path):
                raise TypeError('Unexpected type {} for elem {}'.format(type(elem).__name__, elem))
            return self.evaluate_path(causal_setting, elem)

    def evaluate_situation(self, causal_setting: _ForwardCausalSetting, situation: Situation) -> bool:
        raise NotImplementedError

    def evaluate_path(self, causal_setting: _ForwardCausalSetting, path: Path) -> bool:
        raise NotImplementedError



In [57]:
class StateFormula(Formula):

    def evaluate_path(self, causal_setting: _ForwardCausalSetting, path: Path) -> bool:
        return self.evaluate_situation(causal_setting, path.root)


In [58]:
class PathFormula(Formula):
    pass

In [59]:
@dataclass(order=True, frozen=True)
class CausalSetting:
    fluent_alphabet: FrozenSet[Fluent] = field(default_factory=frozenset)  # without Poss, Know, Int
    action_alphabet: FrozenSet[Action] = field(default_factory=frozenset)
    initial_state: State = field(default_factory=State)
    k_relation: FrozenRelation = field(default_factory=FrozenRelation)
    max_time: int = field(default=0)

    def poss(self, action: Action, situation: Situation) -> bool:
        raise NotImplementedError

    def legal_actions(self, situation: Situation) -> Iterator[Action]:
        return (action for action in self.action_alphabet if self.poss(action, situation))

    def _legal_actions(self, state: State) -> Iterator[Action]:
        return (action for action in self.action_alphabet if self.poss(action, Situation(state)))

    def all_paths(self, situation: Situation) -> Iterator[Path]:
        for path in self._all_paths(situation.state):
            states = [situation.state]
            actions = []
            for sa in path:
                state, action = sa
                states.append(state)
                actions.append(action)
            yield Path(tuple(states), tuple(actions))


    def _all_paths(self, state: State, depth: int = 0) -> Iterator[Sequence[Tuple[State, Action]]]:
        if depth <= self.max_time:
            for action in self._legal_actions(state):
                state_ = self._do(action, state)
                all_paths = self._all_paths(state_, depth + 1)
                for path in all_paths:
                    yield ((state_, action), *path)
        else:
            yield ()

    def do(self, action: Action, situation: Optional[Situation] = None) -> Situation:
        if situation is None:
            state = self.initial_state
            situation = Situation(state)
        else:
            state = situation.state
        state_ = self._do(action, state)
        situation_ = Situation(state_, action, situation)
        return situation_

    def _do(self, action: Action, state: State) -> State:
        if action.symbol == 'req':
            return State(state.world, state.k_relation, (action.arguments[0], *state.prioritized_goals))
        elif action.symbol == 'sense':
            sense: Literal = action.arguments[0]
            return State(state.world, FrozenRelation(frozenset({
                (elem[0], elem[1]) for elem in state.k_relation.relation_set if sense.sign == (abs(sense) in elem[1])
            })), state.prioritized_goals)
        else:
            raise NotImplementedError

    def do_iter(self, actions: Sequence[Action], situation: Optional[Situation] = None) -> Situation:
        for action in actions:
            situation = self.do(action, situation)
        return situation

    def know(self, formula: Formula, situation: Situation) -> bool:
        return all(formula.evaluate(self, situation_) for situation_ in self.k_relation.get_inverse(situation))

    def _g_intersect(self, situation:Situation, level: int) -> Iterator[Path]:
        for s_ in situation.state.k_relation.get_inverse(situation.state.world):
            for p in self.all_paths(s_):
                if self._g_accessible(p, level, situation):
                    yield p

    def _level_intention(self, formula: Formula, situation: Situation, level: int) -> bool:
        return all(formula.evaluate(self, path) for path in self._g_intersect(situation, level))

    def _g_accessible(self, path: Path, level: int, situation: Situation):
        return len(situation.state.prioritized_goals) <= level or situation.state.prioritized_goals[level].evaluate(path)

    def intention(self, formula: Formula, situation: Situation) -> bool:
        return all(self._level_intention(formula, situation, n) for n in range(len(situation.state.prioritized_goals)))

    def causes_directly(self) -> bool:
        pass

    def causes_indirectly(self) -> bool:
        pass

    def causes(self) -> bool:
        return self.causes_directly() or self.causes_indirectly()

In [60]:
@dataclass(order=True, frozen=True)
class Predicate(StateFormula):
    predicate: FluentLiteral

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return self.predicate in situation.state

In [61]:
@dataclass(order=True, frozen=True)
class Possible(StateFormula):
    action: Action

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return causal_setting.poss(self.action, situation)


In [62]:
@dataclass(order=True, frozen=True)
class After(StateFormula):
    action: Action
    formula: Formula

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return self.formula.evaluate(causal_setting, causal_setting.do(self.action, situation))

In [63]:
@dataclass(order=True, frozen=True)
class Negation(Formula):
    formula: Formula

    def evaluate(self, causal_setting: CausalSetting, elem: Union[State, Path]) -> bool:
        return not self.formula.evaluate(causal_setting, elem)

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return not self.formula.evaluate_situation(causal_setting, situation)

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return not self.formula.evaluate_path(causal_setting, path)

    def __neg__(self):
        return self.formula

In [64]:
@dataclass(order=True, frozen=True)
class Conjunction(Formula):
    left: Formula
    right: Formula

    def evaluate(self, causal_setting: CausalSetting, elem: Union[State, Path]) -> bool:
        return self.left.evaluate(causal_setting, elem) and self.right.evaluate(causal_setting, elem)

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return self.left.evaluate_situation(causal_setting, situation) and self.right.evaluate_situation(causal_setting,
                                                                                                         situation)

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return self.left.evaluate_path(causal_setting, path) and self.right.evaluate_path(causal_setting, path)


In [65]:
@dataclass(order=True, frozen=True)
class Disjunction(Formula):
    left: Formula
    right: Formula

    def evaluate(self, causal_setting: CausalSetting, elem: Union[State, Path]) -> bool:
        return self.left.evaluate(causal_setting, elem) or self.right.evaluate(causal_setting, elem)

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return self.left.evaluate_situation(causal_setting, situation) or self.right.evaluate_situation(causal_setting,
                                                                                                        situation)

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return self.left.evaluate_path(causal_setting, path) or self.right.evaluate_path(causal_setting, path)


In [66]:
@dataclass(order=True, frozen=True)
class Know(StateFormula):
    formula: Formula

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return causal_setting.know(self, situation)

In [67]:
@dataclass(order=True, frozen=True)
class Intention(StateFormula):
    formula: Formula

    def evaluate_state(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        raise NotImplementedError


In [68]:
@dataclass(order=True, frozen=True)
class Next(PathFormula):
    formula: Formula

    def evaluate_path(self, causal_setting: _ForwardCausalSetting, path: Path) -> bool:
        for action in causal_setting.action_alphabet:
            if not causal_setting.poss(action, path.root):
                continue
            path_ = path.get_suffix(causal_setting.do(action, Situation(path.root)).state)
            if self.formula.evaluate_path(causal_setting, path_):
                return True
        return False



In [69]:
@dataclass(order=True, frozen=True)
class Until(PathFormula):
    holds_formula: Formula
    until_formula: Formula

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        for s_ in path.states:
            p_ = path.get_suffix(s_)
            if not self.until_formula.evaluate_path(causal_setting, p_):
                continue
            valid = True
            for s_star in path.states:
                p_star = path.get_suffix(s_star)
                if not self.holds_formula.evaluate_path(causal_setting, p_star):
                    valid = False
                    break
            if valid:
                return True
        return False

In [70]:
@dataclass(order=True, frozen=True)
class TrivialTrue(Formula):

    def evaluate(self, causal_setting: CausalSetting, elem: Union[State, Path]) -> bool:
        return True

    def evaluate_situation(self, causal_setting: CausalSetting, situation: Situation) -> bool:
        return True

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return True


In [71]:
@dataclass(order=True, frozen=True)
class Eventually(PathFormula):
    formula: Formula

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return Until(TrivialTrue(), self.formula).evaluate_path(causal_setting, path)

In [72]:
@dataclass(order=True, frozen=True)
class Before(PathFormula):
    before_formula: Formula
    after_formula: Formula

    def evaluate_path(self, causal_setting: CausalSetting, path: Path) -> bool:
        return Negation(Until(Negation(self.before_formula), self.after_formula)).evaluate_path(causal_setting, path)

In [73]:
Ls = Atom('Ls')
Ld = Atom('Ld')
L1 = Atom('L1')
L1_ = Atom("L1'")

takeOff_Ls = Atom('takeOff', (Ls,))
takeOff_Ld = Atom('takeOff', (Ld,))
takeOff_L1 = Atom('takeOff', (L1,))
takeOff_L1_ = Atom('takeOff',(L1_,))

flyTo_Ls = Atom('flyTo', (Ls,))
flyTo_Ld = Atom('flyTo', (Ld,))
flyTo_L1 = Atom('flyTo', (L1,))
flyTo_L1_ = Atom('flyTo', (L1_,))

land_Ls = Atom('land',  (Ls ,))
land_Ld = Atom('land',  (Ld ,))
land_L1 = Atom('land',  (L1 ,))
land_L1_ = Atom('land', (L1_,))

atom_At_Ls = Atom('At', (Ls,))
atom_At_Ld = Atom('At', (Ld,))
atom_At_L1 = Atom('At', (L1,))
atom_At_L1_ = Atom('At', (L1_,))

At_Ls = Literal(atom_At_Ls)
At_Ld = Literal(atom_At_Ld)
At_L1 = Literal(atom_At_L1)
At_L1_ = Literal(atom_At_L1_)

a_Flying = Atom('Flying')

Flying = Literal(a_Flying)

atom_Vis_Ls = Atom('Vis', (Ls,))
atom_Vis_Ld = Atom('Vis', (Ld,))
atom_Vis_L1 = Atom('Vis', (L1,))
atom_Vis_L1_ = Atom('Vis', (L1_,))

Vis_Ls = Literal(atom_Vis_Ls)
Vis_Ld = Literal(atom_Vis_Ld)
Vis_L1 = Literal(atom_Vis_L1)
Vis_L1_ = Literal(atom_Vis_L1_)

atom_TStrom_Ls = Atom('TStrom', (Ls,))
atom_TStrom_Ld = Atom('TStrom', (Ld,))
atom_TStrom_L1 = Atom('TStrom', (L1,))
atom_TStrom_L1_ = Atom('TStrom', (L1_,))

TStrom_Ls = Literal(atom_TStrom_Ls)
TStrom_Ld = Literal(atom_TStrom_Ld)
TStrom_L1 = Literal(atom_TStrom_L1)
TStrom_L1_ = Literal(atom_TStrom_L1_)

sense_TStrom_Ls = Atom('sense', (TStrom_Ls,))
sense_TStrom_Ld = Atom('sense', (TStrom_Ld,))
sense_TStrom_L1 = Atom('sense', (TStrom_L1,))
sense_Tstrom_L1_ = Atom('sense', (TStrom_L1_,))

req_Eventually_Vis_L1_ = Atom('req', (Eventually(Vis_L1_),))

route = {
    Ls: frozenset({L1, L1_}),
    L1: frozenset({Ld}),
    L1_: frozenset({Ld}),
    Ld: frozenset()
}

drone_example_actions = {takeOff_Ls, takeOff_L1, takeOff_L1_, takeOff_Ld,
                         land_Ls, land_L1, land_L1_, land_Ld,
                         flyTo_Ls, flyTo_L1, flyTo_L1_, flyTo_Ld,
                         sense_TStrom_Ls, sense_TStrom_Ld, sense_TStrom_L1, sense_Tstrom_L1_,
                         req_Eventually_Vis_L1_}

drone_example_fluents = {
    atom_At_Ls, atom_At_L1, atom_At_L1_, atom_At_Ld,
    atom_Vis_Ls, atom_At_L1, atom_At_L1_, atom_At_Ld,
    atom_TStrom_Ls, atom_TStrom_L1, atom_TStrom_L1_, atom_TStrom_Ld
}

In [74]:
@dataclass(order=True, frozen=True)
class DroneExample(CausalSetting):

    def poss(self, action: Action, situation: Situation) -> bool:
        if action not in self.action_alphabet:
            return False
        if action.symbol == 'sense':
            return True
        elif action.symbol == 'req':
            return True
        l = action.arguments[0]
        atom_At_l = Atom('At', l)
        At_l = Literal(atom_At_l)
        if At_l not in situation.state:
            return False
        if action.symbol == 'takeOff':
            return -Flying in situation.state
        if action.symbol == 'land':
            return Flying in situation.state
        l_ = action.arguments[1]
        if action.symbol == 'flyTo':
            return Flying in situation.state and l_ in route[l]

        assert False, "Unexpected action {}".format(action)

    def _do(self, action: Action, state: State) -> State:
        if action.symbol == 'takeOff':
            def update(s):
                return (s - {-Flying}) | {Flying}
        elif action.symbol == 'land':
            def update(s):
                return (s - {Flying}) | {-Flying}
        elif action.symbol == 'flyTo':
            l = action.arguments[0]
            l_ = action.arguments[1]
            atom_At_l = Atom('At', l)
            atom_At_l_ = Atom('At', l_)
            At_l = Literal(atom_At_l)
            At_l_ = Literal(atom_At_l_)

            def update(s):
                return (s - {At_l}) | {-At_l, At_l_}
        else:
            return super(DroneExample, self)._do(action, state)
        return State(update(state.world), FrozenRelation(
            frozenset({(update(elem[0]), update(elem[1])) for elem in state.k_relation.relation_set})), state.prioritized_goals)





In [75]:
drone_example_initial_world = frozenset({
    Vis_Ls, At_Ls, TStrom_L1
})
print(drone_example_initial_world)

frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)})


In [76]:
drone_example_k_states = {frozenset(Literal(atom) for atom in atoms) for atoms in
                          more_itertools.powerset(drone_example_fluents)}
len(drone_example_k_states)

512

In [77]:
drone_example_valid_k_states = {
    k_state for k_state in drone_example_k_states if
    At_Ls in k_state and
    At_L1 not in k_state and
    At_L1_ not in k_state and
    At_Ld not in k_state and
    Flying not in k_state and
    Vis_Ls in k_state and
    Vis_L1 not in k_state and
    Vis_L1_ not in k_state and
    Vis_Ld not in k_state and
    TStrom_Ls not in k_state and
    TStrom_L1_ not in k_state and
    TStrom_Ld not in k_state
}
len(drone_example_valid_k_states)

2

In [78]:
for k_state in drone_example_valid_k_states:
    print(k_state)

frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)})
frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True)})


In [79]:
drone_example_k_relation_ground_coll = {
    (drone_example_initial_world, k_state) for k_state in drone_example_valid_k_states
}
drone_example_k_relation = FrozenRelation.from_ground_coll(drone_example_k_relation_ground_coll)
print(drone_example_k_relation)

{(frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}),frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True)})),(frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}),frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', ar

In [80]:
drone_example_phi_0 = Eventually(At_Ld)
drone_example_phi_1 = Before(Vis_L1, Vis_Ld)

In [81]:
drone_example_initial_state = State(frozenset(drone_example_initial_world), drone_example_k_relation, (drone_example_phi_0, drone_example_phi_1))
print(drone_example_initial_state)

State(world=frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), k_relation=FrozenRelation(relation_set=frozenset({(frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True)})), (frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arg

In [82]:
-Flying

Literal(atom=Atom(symbol='Flying', arguments=()), sign=False)

In [83]:
(-Flying) in drone_example_initial_state

True

In [84]:
Vis_Ls in drone_example_initial_state

True

In [85]:
drone_example = DroneExample(
    drone_example_fluents,
    drone_example_actions,
    drone_example_initial_state,
    drone_example_k_relation,
    7,
)

In [86]:
drone_example_s3 = drone_example.do_iter([takeOff_Ls, sense_TStrom_L1, req_Eventually_Vis_L1_])
print(len(drone_example_s3.parent.parent.parent.state.k_relation.relation_set))
drone_example_s3.parent.parent.parent

2


Situation(state=State(world=frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), k_relation=FrozenRelation(relation_set=frozenset({(frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True)})), (frozenset({Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(

In [87]:
print(drone_example_s3.parent.parent.action)
print(len(drone_example_s3.parent.parent.state.k_relation.relation_set))
drone_example_s3.parent.parent

takeOff(Ls)
2


Situation(state=State(world=frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), k_relation=FrozenRelation(relation_set=frozenset({(frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls

In [88]:
print(drone_example_s3.parent.action)
print(len(drone_example_s3.parent.state.k_relation.relation_set))
drone_example_s3.parent

sense(TStrom(L1))
1


Situation(state=State(world=frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), k_relation=FrozenRelation(relation_set=frozenset({(frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls

In [89]:
print(drone_example_s3.action)
print(len(drone_example_s3.state.k_relation.relation_set))
drone_example_s3

req(Eventually(formula=Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol="L1'", arguments=()),)), sign=True)))
1


Situation(state=State(world=frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), k_relation=FrozenRelation(relation_set=frozenset({(frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='TStrom', arguments=(Atom(symbol='L1', arguments=()),)), sign=True)}), frozenset({Literal(atom=Atom(symbol='Flying', arguments=()), sign=True), Literal(atom=Atom(symbol='At', arguments=(Atom(symbol='Ls', arguments=()),)), sign=True), Literal(atom=Atom(symbol='Vis', arguments=(Atom(symbol='Ls

In [90]:
sum(1 for _ in drone_example.all_paths(Situation(drone_example_initial_state)))

390625

In [91]:
drone_example.intention(drone_example_phi_0, drone_example_s3)

AttributeError: 'frozenset' object has no attribute 'state'

In [ ]:
drone_example.intention(drone_example_phi_1, drone_example_s3)